In [1]:
%reload_ext autoreload
%autoreload 2
import dpu_clf
import gym
import dotenv
dotenv.load_dotenv()

c:\Users\matan\anaconda3\envs\procgen_env_clone\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
seed = 8798

env_kwargs = {
    "render_mode": "rgb_array",
    "num_levels": 0,
    "start_level": 0,
    "distribution_mode": "easy",
    "rand_seed": seed,
    "use_discrete_action_wrapper": True, 
    "use_stay_bonus_wrapper": False,
    'food_diversity': 4,
    'fruitbot_num_walls': 5,
    'fruitbot_num_good_min': 5,
    'fruitbot_num_good_range': 1,
    'fruitbot_num_bad_min': 5,
    'fruitbot_num_bad_range': 1,
    'fruitbot_wall_gap_pct': 50,
    'fruitbot_door_prob_pct': 0,
    "fruitbot_reward_positive": 2,
    "fruitbot_reward_negative": -1,
    "fruitbot_reward_wall_hit": -2,
    }


env = gym.make("procgen:procgen-fruitbot-v0", **env_kwargs)

# model_path = "models\\fruitbot\\20251214-084655_easy\\ppo_final.zip"
# model_path2 = "models\\fruitbot\\20251213-212435_easy\\ppo_final.zip"
# agent = dpu_clf.load_agent(None, model_path2)

Using prebuilt binaries from: c:\Users\matan\master_thesis\rl_envs\procgen\procgen\.build\relwithdebinfo\RelWithDebInfo


In [3]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from stable_baselines3 import PPO
from IPython.display import display
from PIL import Image

## Detailed Episode Data

You can access detailed per-episode data for each model:

In [4]:
# Find all models in the fruitbot folder
models_base_dir = Path("models/fruitbot")
model_dirs = sorted([d for d in models_base_dir.iterdir() if d.is_dir()])

print(f"Found {len(model_dirs)} model directories:")
for model_dir in model_dirs:
    model_file = model_dir / "ppo_final.zip"
    exists = "✓" if model_file.exists() else "✗"
    print(f"  {exists} {model_dir.name}")

Found 16 model directories:
  ✓ 20251130-001800_easy
  ✓ 20251201-110009_easy
  ✓ 20251215-094437_easy
  ✓ 20251222-194508_easy
  ✓ 20251223-133810_easy
  ✓ 20251224-140335_D2_easy
  ✓ 20251225-155401_hard
  ✓ 20251226-014845_hard
  ✓ 20251227-205223_hard
  ✓ 20251230-134845_easy
  ✓ 20251231-174002_easy
  ✗ 20260101-112255_easy
  ✓ 20260101-112929_easy
  ✓ 20260101-150436_easy
  ✓ 20260101-165959_easy
  ✗ 20260103-073446_easy


In [5]:
def evaluate_model(model_path, env, num_episodes=10, 
                   fruitbot_reward_positive=2.0,
                   fruitbot_reward_negative=-1.0,
                   fruitbot_reward_wall_hit=-2.0):
    """
    Evaluate a single model and return episode statistics.;
    
    Returns:
        dict with per-episode data and summary statistics
    """
    # Load model
    model = PPO.load(model_path)
    
    # Track per-episode data
    episodes_data = []
    
    for ep in range(num_episodes):
        obs = env.reset()
        if isinstance(obs, tuple):
            obs, _ = obs
        
        done = False
        truncated = False
        total_reward = 0.0
        steps = 0
        good_food = 0
        bad_food = 0
        wall_hits = 0
        first_frame = True
        while not (done or truncated):
            action, _states = model.predict(obs, deterministic=True)
            result = env.step(action)
            
            # Handle both gym and gymnasium API
            if len(result) == 4:
                obs, reward, done, info = result
                truncated = False
            else:
                obs, reward, done, truncated, info = result

            # if first_frame:
            #     game_image = info.get('rgb', None)
            #     game_image = Image.fromarray(game_image).convert('RGB').resize((512, 512))
            #     display(game_image)
            #     first_frame = False

            
            # Ensure reward is a scalar float
            try:
                r = float(np.asarray(reward).item())
            except Exception:
                r = float(reward)
            
            total_reward += r
            steps += 1
            # Count events based on reward values
            TOL = 1e-3
            if np.isclose(r, fruitbot_reward_positive, atol=TOL, rtol=0.0):
                good_food += 1
            elif np.isclose(r, fruitbot_reward_negative, atol=TOL, rtol=0.0):
                bad_food += 1
            elif np.isclose(r, fruitbot_reward_wall_hit, atol=TOL, rtol=0.0):
                wall_hits += 1
        
        episodes_data.append({
            'episode': ep + 1,
            'reward': total_reward,
            'steps': steps,
            'good_food': good_food,
            'bad_food': bad_food,
            'wall_hits': wall_hits
        })
    
    # Calculate summary statistics
    rewards = [ep['reward'] for ep in episodes_data]
    lengths = [ep['steps'] for ep in episodes_data]
    good_foods = [ep['good_food'] for ep in episodes_data]
    bad_foods = [ep['bad_food'] for ep in episodes_data]
    wall_hits_list = [ep['wall_hits'] for ep in episodes_data]
    
    summary = {
        'mean_reward': np.mean(rewards),
        'std_reward': np.std(rewards),
        'mean_steps': np.mean(lengths),
        'mean_good_food': np.mean(good_foods),
        'mean_bad_food': np.mean(bad_foods),
        'mean_wall_hits': np.mean(wall_hits_list),
        'episodes_data': episodes_data
    }
    
    return summary

In [6]:
#create the environment
seed = 8798
# Use the reward values from your env_kwargs or set them explicitly
fruitbot_reward_positive = 2.0
fruitbot_reward_negative = -2.0
fruitbot_reward_wall_hit = -5.0

env_kwargs = {
    "render_mode": "rgb_array",
    "num_levels": 0,
    "start_level": 0,
    "distribution_mode": "easy",
    "rand_seed": 0,
    "use_discrete_action_wrapper": True, 
    "use_stay_bonus_wrapper": False,
    'food_diversity': 4,
    'fruitbot_num_walls': 5,
    'fruitbot_num_good_min': 5,
    'fruitbot_num_good_range': 1,
    'fruitbot_num_bad_min': 5,
    'fruitbot_num_bad_range': 1,
    'fruitbot_wall_gap_pct': 50,
    'fruitbot_door_prob_pct': 10,
    "fruitbot_reward_positive": fruitbot_reward_positive,
    "fruitbot_reward_negative": fruitbot_reward_negative,
    "fruitbot_reward_wall_hit": fruitbot_reward_wall_hit,
    }

hard_env_kwargs = env_kwargs.copy()
hard_env_kwargs['distribution_mode'] = 'hard'


env_easy = gym.make("procgen-fruitbot-v0", **env_kwargs)
env_hard = gym.make("procgen-fruitbot-v0", **hard_env_kwargs)


# Evaluate all models
results = []



for i, model_dir in enumerate(model_dirs):
    model_file = model_dir / "ppo_final.zip"
    
    if not model_file.exists():
        print(f"Skipping {model_dir.name} - model file not found")
        continue
    
    try:
        print(f"\n[{i+1}/{len(model_dirs)}] Evaluating {model_dir.name}...")
        env = env_easy if 'easy' in model_dir.name else env_hard
        # Evaluate model
        eval_results = evaluate_model(
            model_path=str(model_file).replace('.zip', ''),  # PPO.load adds .zip automatically
            env=env,
            num_episodes=30,
            fruitbot_reward_positive=fruitbot_reward_positive,
            fruitbot_reward_negative=fruitbot_reward_negative,
            fruitbot_reward_wall_hit=fruitbot_reward_wall_hit
        )
        
        # Store summary results
        results.append({
            'model_name': model_dir.name,
            'mean_reward': eval_results['mean_reward'],
            'std_reward': eval_results['std_reward'],
            'mean_steps': eval_results['mean_steps'],
            'mean_good_food': eval_results['mean_good_food'],
            'mean_bad_food': eval_results['mean_bad_food'],
            'mean_wall_hits': eval_results['mean_wall_hits'],
            'episodes_data': eval_results['episodes_data']
        })
        
        print(f"  Mean reward: {eval_results['mean_reward']:.2f} ± {eval_results['std_reward']:.2f}")
        print(f"  Mean steps: {eval_results['mean_steps']:.1f}")
        print(f"  Mean good food: {eval_results['mean_good_food']:.2f}")
        print(f"  Mean bad food: {eval_results['mean_bad_food']:.2f}")
        print(f"  Mean wall hits: {eval_results['mean_wall_hits']:.2f}")
        
    except Exception as e:
        print(f"Error evaluating {model_dir.name}: {e}")
        continue

print(f"\n{'='*60}")
print(f"Evaluation complete! Evaluated {len(results)} models.")
print(f"{'='*60}")

Using prebuilt binaries from: c:\Users\matan\master_thesis\rl_envs\procgen\procgen\.build\relwithdebinfo\RelWithDebInfo
Using prebuilt binaries from: c:\Users\matan\master_thesis\rl_envs\procgen\procgen\.build\relwithdebinfo\RelWithDebInfo

[1/16] Evaluating 20251130-001800_easy...
  Mean reward: 16.27 ± 8.66
  Mean steps: 130.7
  Mean good food: 1.87
  Mean bad food: 1.53
  Mean wall hits: 0.07

[2/16] Evaluating 20251201-110009_easy...
  Mean reward: 9.60 ± 10.50
  Mean steps: 120.7
  Mean good food: 2.03
  Mean bad food: 1.90
  Mean wall hits: 0.27

[3/16] Evaluating 20251215-094437_easy...
  Mean reward: 17.07 ± 11.15
  Mean steps: 126.8
  Mean good food: 2.70
  Mean bad food: 1.17
  Mean wall hits: 0.13

[4/16] Evaluating 20251222-194508_easy...
  Mean reward: 15.00 ± 10.53
  Mean steps: 114.7
  Mean good food: 2.27
  Mean bad food: 1.53
  Mean wall hits: 0.20

[5/16] Evaluating 20251223-133810_easy...
  Mean reward: 9.43 ± 11.70
  Mean steps: 107.2
  Mean good food: 1.70
  Mean b

In [7]:
# Create summary DataFrame
summary_df = pd.DataFrame([
    {
        'model_name': r['model_name'],
        'mean_reward': r['mean_reward'],
        'std_reward': r['std_reward'],
        'mean_steps': r['mean_steps'],
        'mean_good_food': r['mean_good_food'],
        'mean_bad_food': r['mean_bad_food'],
        'mean_wall_hits': r['mean_wall_hits']
    }
    for r in results
])

# Sort by mean reward (descending)
summary_df = summary_df.sort_values('mean_reward', ascending=False)

print("Summary of all models (sorted by mean reward):")
print(summary_df.to_string(index=False))

# Save to CSV
summary_df.to_csv('model_evaluation_summary.csv', index=False)
print("\nResults saved to 'model_evaluation_summary.csv'")

Summary of all models (sorted by mean reward):
             model_name  mean_reward  std_reward  mean_steps  mean_good_food  mean_bad_food  mean_wall_hits
   20251227-205223_hard    21.966667    9.492043  123.900000        3.500000       0.266667        0.100000
   20251225-155401_hard    21.766667    8.781357  125.800000        3.866667       0.400000        0.100000
   20251231-174002_easy    18.133333   12.203096  113.466667        3.866667       0.500000        0.233333
   20251230-134845_easy    17.700000   10.517129  121.366667        3.233333       1.300000        0.166667
20251224-140335_D2_easy    17.433333   10.607178  122.633333        3.433333       1.900000        0.166667
   20251215-094437_easy    17.066667   11.153276  126.800000        2.700000       1.166667        0.133333
   20251130-001800_easy    16.266667    8.663846  130.733333        1.866667       1.533333        0.066667
   20260101-150436_easy    16.166667    7.962761  128.933333        3.700000       3.9333

In [8]:
# Example: View detailed episodes for the best model
if results:
    best_model = summary_df.iloc[0]['model_name']
    best_model_data = next(r for r in results if r['model_name'] == best_model)
    
    print(f"Detailed episodes for best model: {best_model}\n")
    episodes_df = pd.DataFrame(best_model_data['episodes_data'])
    print(episodes_df.head(10))
    
    # Save detailed data for all models
    all_episodes = []
    for r in results:
        for ep_data in r['episodes_data']:
            all_episodes.append({
                'model_name': r['model_name'],
                **ep_data
            })
    
    all_episodes_df = pd.DataFrame(all_episodes)
    all_episodes_df.to_csv('model_evaluation_detailed.csv', index=False)
    print(f"\nDetailed episode data saved to 'model_evaluation_detailed.csv'")
    print(f"Total episodes: {len(all_episodes_df)}")

Detailed episodes for best model: 20251227-205223_hard

   episode  reward  steps  good_food  bad_food  wall_hits
0        1     1.0     80          3         0          1
1        2    28.0    132          4         0          0
2        3     8.0    132          1         2          0
3        4    28.0    132          4         0          0
4        5    28.0    132          4         0          0
5        6    28.0    132          4         0          0
6        7    26.0    132          3         0          0
7        8    28.0    132          4         0          0
8        9    30.0    132          5         0          0
9       10    -5.0     22          0         0          1

Detailed episode data saved to 'model_evaluation_detailed.csv'
Total episodes: 420
